<a href="https://colab.research.google.com/github/vignesh99/CS6300-Speech-Technology/blob/master/HMMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
                                    #Get data files onto root
%cd
from google.colab import drive
drive.mount('/content/gdrive')
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/1 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/2 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/3 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/6 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/7 /root
!cp -avr /content/gdrive/My\ Drive/HMM_CtsData/dev /root
!cp -avr /content/gdrive/My\ Drive/HMM_CtsData/test /root
!cp -avr /content/gdrive/My\ Drive/fp1.txt /root
!cp -avr /content/gdrive/My\ Drive/fp2.txt /root
!cp -avr /content/gdrive/My\ Drive/fp3.txt /root
!cp -avr /content/gdrive/My\ Drive/fp6.txt /root
!cp -avr /content/gdrive/My\ Drive/fp7.txt /root
!ls

/root
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
removed '/root/1'
'/content/gdrive/My Drive/HMM_IsoData/1' -> '/root/1'
removed '/root/2'
'/content/gdrive/My Drive/HMM_IsoData/2' -> '/root/2'
removed '/root/3'
'/content/gdrive/My Drive/HMM_IsoData/3' -> '/root/3'
removed '/root/6'
'/content/gdrive/My Drive/HMM_IsoData/6' -> '/root/6'
removed '/root/7'
'/content/gdrive/My Drive/HMM_IsoData/7' -> '/root/7'
removed '/root/dev/team15'
'/content/gdrive/My Drive/HMM_CtsData/dev/team15' -> '/root/dev/team15'
removed '/root/test/team15'
'/content/gdrive/My Drive/HMM_CtsData/test/team15' -> '/root/test/team15'
'/content/gdrive/My Drive/fp1.txt' -> '/root/fp1.txt'
'/content/gdrive/My Drive/fp2.txt' -> '/root/fp2.txt'
'/content/gdrive/My Drive/fp3.txt' -> '/root/fp3.txt'
'/content/gdrive/My Drive/fp6.txt' -> '/root/fp6.txt'
'/content/gdrive/My Drive/fp7.txt' -> '/root/fp7.txt'
1  6	DHMMtest  fp3.txt  one.seq 

In [90]:
                                    #Test file
!cp -avr /root/1/dev/ra_1.mfcc /root
filex = "ra_1.mfcc"

'/root/1/dev/ra_1.mfcc' -> '/root/ra_1.mfcc'


In [87]:
                                    #Import required libraries
from pylab import *
import os                           #Going through folders
from scipy.cluster.vq import kmeans2
from google.colab import files
from sklearn.cluster import KMeans

**Discete HMMs**

In [88]:
def fileTofeature(fileMfcc) :
                                    #Convert features from file into array
    features = np.genfromtxt(fileMfcc,delimiter='\n',dtype=str)
    Nc,Nf = features[0].split(" ")
    Nc = int(Nc)
    Nf = int(Nf)
    features = np.genfromtxt(features[1:],delimiter=' ',dtype=str)
    features = features.astype(float)
                                    
    return Nf,features


In [91]:
label,ft = fileTofeature(filex)
label

121

In [109]:
                                    #Give the file with sequences for training data
def trainCentre(Nclust=16,folders=None) :
    if folders is None :            #Go through all digits
        folders = array(["1","2","3","6","7"])

    ind = []                     #All digits total vector numbers
    ind.append([0,0])            #Inital values
    Nwin = []                       #No. of vectors in each train file
    count = 0
    for word in folders :
                                    #Go through required folders
        for subdir, dirs, files in os.walk(word):
            dirname = subdir.split(os.path.sep)[-1]
            print('Directory:', dirname)
            if dirname == "train":
                for f in files:
                    if f.endswith('.mfcc'):
                                    #Copy file (mfcc) to root and perform K-means
                        !cp -avr /root/$word/train/$f /root
                        foo,temp = fileTofeature(f)
                                    #Concatenate training feature vectors
                        if count == 0 :
                            features = temp
                            Nwin.append(foo)
                        else :
                            features = np.concatenate((features,temp))
                            Nwin.append(foo+Nwin[-1])
                        !rm $f
                        count+=1
        ind.append([len(Nwin),Nwin[-1]])
                                    #Apply K-means on the vectors
    trainKM = KMeans(n_clusters=Nclust)
    trainKM.fit(features)
                                    #Get Nwin in required format
    ind = array(ind)
    Nwin = array(Nwin)
    for i in range(5) :
        Nwin[ind[i,0]:ind[i+1,0]] -=  ind[i,1]

    return Nwin,features,trainKM,ind


In [110]:
def seqDwnld(seq,fileWord) :
    with open(fileWord,"w") as f:
        f.write("\n".join(" ".join(map(str, line)) for line in seq))

    return None

In [111]:
                                    #Give the file with sequences for training data
def trainSeq(Nwin,features,trainKM,fileWord="check.txt") :
    seq = []
    count = 0
                                    #Go through required folders
    for i in range(0,len(Nwin)) :
                                    #Get each features from all-features-array
        if i != 0:
            temp = features[Nwin[i-1]:Nwin[i]]
        else : 
            temp = features[:Nwin[0]]
                                    #Perform Kmeans on predefined centres
        label = trainKM.predict(temp)
        seq.append(label)

        if i==2 :
            print(label)

    seq = array(seq)

    seqDwnld(seq,fileWord)

    return None


In [102]:
!ls

1  7	      fp1.txt  fp7.txt	  seven.seq   test	  two.test
2  bh_1.mfcc  fp2.txt  one.seq	  seven.test  three.seq
3  dev	      fp3.txt  one.test   six.seq     three.test
6  ek_7.mfcc  fp6.txt  ra_1.mfcc  six.test    two.seq


In [112]:
Nw,feat,cenKM,ind = trainCentre()       #Train all the phenomes

                                    #Get file sequences
trainSeq(Nw[ind[0,0]:ind[1,0]],feat[ind[0,1]:ind[1,1]],cenKM,"one.seq")
trainSeq(Nw[ind[1,0]:ind[2,0]],feat[ind[1,1]:ind[2,1]],cenKM,"two.seq")
trainSeq(Nw[ind[2,0]:ind[3,0]],feat[ind[2,1]:ind[3,1]],cenKM,"three.seq")
trainSeq(Nw[ind[3,0]:ind[4,0]],feat[ind[3,1]:ind[4,1]],cenKM,"six.seq")
trainSeq(Nw[ind[4,0]:ind[5,0]],feat[ind[4,1]:ind[5,1]],cenKM,"seven.seq")


Directory: 1
Directory: train
'/root/1/train/ac_1.mfcc' -> '/root/ac_1.mfcc'
'/root/1/train/ag_1.mfcc' -> '/root/ag_1.mfcc'
'/root/1/train/ai_1.mfcc' -> '/root/ai_1.mfcc'
'/root/1/train/an_1.mfcc' -> '/root/an_1.mfcc'
'/root/1/train/bh_1.mfcc' -> '/root/bh_1.mfcc'
'/root/1/train/bi_1.mfcc' -> '/root/bi_1.mfcc'
'/root/1/train/br_1.mfcc' -> '/root/br_1.mfcc'
'/root/1/train/ca_1.mfcc' -> '/root/ca_1.mfcc'
'/root/1/train/cg_1.mfcc' -> '/root/cg_1.mfcc'
'/root/1/train/cl_1.mfcc' -> '/root/cl_1.mfcc'
'/root/1/train/cm_1.mfcc' -> '/root/cm_1.mfcc'
'/root/1/train/dc_1.mfcc' -> '/root/dc_1.mfcc'
'/root/1/train/dg_1.mfcc' -> '/root/dg_1.mfcc'
'/root/1/train/ea_1.mfcc' -> '/root/ea_1.mfcc'
'/root/1/train/ec_1.mfcc' -> '/root/ec_1.mfcc'
'/root/1/train/ee_1.mfcc' -> '/root/ee_1.mfcc'
'/root/1/train/eg_1.mfcc' -> '/root/eg_1.mfcc'
'/root/1/train/ei_1.mfcc' -> '/root/ei_1.mfcc'
'/root/1/train/ek_1.mfcc' -> '/root/ek_1.mfcc'
'/root/1/train/es_1.mfcc' -> '/root/es_1.mfcc'
'/root/1/train/hg_1.mfcc' -> '

In [114]:
                                    #Download the train sequence files
files.download("one.seq")
files.download("two.seq")
files.download("three.seq")
files.download("six.seq")
files.download("seven.seq")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [113]:
!ls
print(len(feat))
Nw

1  6	fp1.txt  fp6.txt  one.test   seven.test  test	     two.seq
2  7	fp2.txt  fp7.txt  ra_1.mfcc  six.seq	 three.seq   two.test
3  dev	fp3.txt  one.seq  seven.seq  six.test	 three.test
19968


array([  75,  159,  238,  322,  423,  520,  636,  730,  831,  925, 1028,
       1222, 1330, 1418, 1503, 1592, 1662, 1759, 1856, 1948, 2045, 2141,
       2238, 2359, 2444, 2538, 2646, 2721, 2818, 2910, 3034, 3146, 3224,
       3305, 3412, 3496, 3606, 3684, 3769,   70,  153,  233,  316,  422,
        525,  645,  743,  830,  937, 1038, 1126, 1205, 1289, 1372, 1444,
       1575, 1662, 1737, 1825, 1942, 2034, 2132, 2230, 2308, 2432, 2517,
       2601, 2677, 2770, 2876, 2969, 3068, 3153, 3256, 3343, 3456, 3534,
       3631,   71,  159,  242,  349,  452,  540,  646,  754,  843, 1024,
       1154, 1247, 1330, 1411, 1494, 1577, 1671, 1764, 1865, 1953, 2047,
       2159, 2275, 2458, 2538, 2621, 2733, 2829, 2930, 3024, 3148, 3254,
       3335, 3422, 3526, 3615, 3707, 3792, 3898,   97,  207,  341,  458,
        565,  696,  825,  946, 1061, 1169, 1309, 1435, 1548, 1645, 1761,
       1872, 1966, 2076, 2178, 2317, 2439, 2561, 2722, 2837, 2936, 3053,
       3161, 3269, 3377, 3490, 3635, 3760, 3876, 39

In [115]:
def testKmeans(fileMfcc,trainKM,Nclust=16,iters=1) :
                                    #Convert features from file into array
    features = np.genfromtxt(fileMfcc,delimiter='\n',dtype=str)
    Nc,Nf = features[0].split(" ")
    Nc = int(Nc)
    Nf = int(Nf)
    features = np.genfromtxt(features[1:],delimiter=' ',dtype=str)
    features = features.astype(float)
    
                                    #Apply K-means on the vectors
    if len(features) == Nf and len(features[0]) == Nc :
        label = trainKM.predict(features)

    return label


In [116]:
                                    #Give the file with sequences for testing data
def testSeq(trainKM,fileWord="check.txt",folders=None) :
    if folders is None :
        folders = array(["1","2","3","6","7"])
    seq = []
    count = np.zeros(5)
    for word in folders :    
                                        #Go through required folders
        for subdir, dirs, files in os.walk(word):
            dirname = subdir.split(os.path.sep)[-1]
            print('Directory:', dirname)
            if dirname == "dev":
                for f in files:
                    if f.endswith(".mfcc"):
                                        #Copy file (mfcc) to root and perform K-means
                        !cp -avr /root/$word/dev/$f /root
                        label = testKmeans(f,trainKM)
                        seq.append(label)
                        !rm $f
                        count[np.where(word == folders)[0][0]] += 1
        if word != "1" :
            count[np.where(word == folders)[0][0]] = count[np.where(word == folders)[0][0]] + count[np.where(word == folders)[0][0]-1]

    print(count)
    seq = array(seq)
    
    seqDwnld(seq,fileWord)

    return count


In [117]:
                                    #Get the test sequence
count = testSeq(cenKM,"DHMMtest")

Directory: 1
Directory: train
Directory: dev
'/root/1/dev/mk_1.mfcc' -> '/root/mk_1.mfcc'
'/root/1/dev/mm_1.mfcc' -> '/root/mm_1.mfcc'
'/root/1/dev/ms_1.mfcc' -> '/root/ms_1.mfcc'
'/root/1/dev/mw_1.mfcc' -> '/root/mw_1.mfcc'
'/root/1/dev/nc_1.mfcc' -> '/root/nc_1.mfcc'
'/root/1/dev/ng_1.mfcc' -> '/root/ng_1.mfcc'
'/root/1/dev/nh_1.mfcc' -> '/root/nh_1.mfcc'
'/root/1/dev/pe_1.mfcc' -> '/root/pe_1.mfcc'
'/root/1/dev/pk_1.mfcc' -> '/root/pk_1.mfcc'
'/root/1/dev/pm_1.mfcc' -> '/root/pm_1.mfcc'
'/root/1/dev/pp_1.mfcc' -> '/root/pp_1.mfcc'
'/root/1/dev/ra_1.mfcc' -> '/root/ra_1.mfcc'
Directory: 2
Directory: train
Directory: dev
'/root/2/dev/mk_2.mfcc' -> '/root/mk_2.mfcc'
'/root/2/dev/mm_2.mfcc' -> '/root/mm_2.mfcc'
'/root/2/dev/ms_2.mfcc' -> '/root/ms_2.mfcc'
'/root/2/dev/mw_2.mfcc' -> '/root/mw_2.mfcc'
'/root/2/dev/nc_2.mfcc' -> '/root/nc_2.mfcc'
'/root/2/dev/ng_2.mfcc' -> '/root/ng_2.mfcc'
'/root/2/dev/nh_2.mfcc' -> '/root/nh_2.mfcc'
'/root/2/dev/pe_2.mfcc' -> '/root/pe_2.mfcc'
'/root/2/d

In [118]:
                                    #Download the test sequence files
files.download("DHMMtest")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [120]:
def testProb(fileProb) :
    prob = np.genfromtxt(fileProb,delimiter=' ',dtype=str)
    prob = prob.astype(float)

    return prob

In [121]:
!ls

1  6	DHMMtest  fp3.txt  one.seq    seven.test  test	      two.seq
2  7	fp1.txt   fp6.txt  one.test   six.seq	  three.seq   two.test
3  dev	fp2.txt   fp7.txt  seven.seq  six.test	  three.test


In [122]:
def accModel(count) :

    error = np.zeros(5)
    prob1 = testProb("fp1.txt")
    allprob = np.zeros((len(prob1),5))
    allprob[:,0] = prob1
    
    allprob[:,1] = testProb("fp2.txt")
    allprob[:,2] = testProb("fp3.txt")
    allprob[:,3] = testProb("fp6.txt")
    allprob[:,4] = testProb("fp7.txt")
    
    estimate = np.argmax(allprob,axis=1)
    print(estimate)
    for i in range(0,5) :
        if i==0:
            error[i] = len(np.where(estimate[:count[i]]!=i)[0])
        else :
            error[i] = len(np.where(estimate[count[i-1]:count[i]]!=i)[0])

    print(error)
    acc = 1-(sum(error)/count[-1])
    
    return acc

In [123]:
#acc = accModel(array([12,24,36,48,60]))
acc = accModel(count.astype(int))
print(acc)

[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1 4 2 2 2 2 2 2 2 2 2 2 2 2 3
 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 1 4 4 4 4 1 4]
[0. 2. 0. 0. 2.]
0.9333333333333333


**Continuous HMMs**

In [ ]:
                                    #Concatanate 2-phoneme models
def 